In [4]:
import os
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import clear_output  # For clearing output in Jupyter/Colab
import os
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import clear_output  # For clearing output in Jupyter/Colab

# ============================
# Global Parameters & Paths
# ============================
SR = 32000
DURATION = 5.0
BATCH_SIZE = 8
EPOCHS = 100  # increased epochs to see the effect of patience.
N_MELS = 56
N_MFCC = 42
patience = 5  # Added patience variable
TIME_STRETCH_RATE = 0.1  # Percentage to stretch/compress time
PITCH_SHIFT_SEMITONES = 2  # Maximum semitones to shift pitch
NOISE_LEVEL = 0.005  # Level of random noise to add
DROPOUT_RATE = 0.2  # Added dropout rate.
alpha = 0.01

BASE_DIR = r"C:\BIRD_DATA"
TRAIN_AUDIO_DIR = os.path.join(BASE_DIR, "train_audio")
TRAIN_CSV_PATH = os.path.join(BASE_DIR, "train.csv")

# ============================
# Load Metadata and Sample 1%
# ============================
train_df = pd.read_csv(TRAIN_CSV_PATH)

# Sample 1% of the data
sampled_df = train_df.sample(frac=0.01, random_state=42)

species_codes = sorted(sampled_df["primary_label"].unique())
species_to_index = {code: i for i, code in enumerate(species_codes)}
num_species = len(species_codes)

# ============================
# Audio Processing Functions with Augmentation
# ============================
def load_and_process_audio(path, sr=SR, duration=DURATION, n_mels=N_MELS, n_mfcc=N_MFCC, target_length=313,
                           time_stretch_rate=TIME_STRETCH_RATE, pitch_shift_semitones=PITCH_SHIFT_SEMITONES, noise_level=NOISE_LEVEL):
    try:
        audio, _ = librosa.load(path, sr=sr, duration=duration)

        # Apply Data Augmentation (Randomly)
        if np.random.rand() < 0.5:  # 50% chance of applying augmentation
            # Time Stretching
            if np.random.rand() < 0.3:  # 30% chance of time stretching
                rate = np.random.uniform(1.0 - time_stretch_rate, 1.0 + time_stretch_rate)
                audio = librosa.effects.time_stretch(audio, rate=rate)

            # Pitch Shifting
            if np.random.rand() < 0.3:  # 30% chance of pitch shifting
                n_steps = np.random.randint(-pitch_shift_semitones, pitch_shift_semitones + 1)
                audio = librosa.effects.pitch_shift(audio, sr=sr, n_steps=n_steps)

            # Adding Noise
            if np.random.rand() < 0.3:  # 30% chance of adding noise
                noise = np.random.randn(len(audio)) * noise_level
                audio = audio + noise

        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
        mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spectrogram), n_mfcc=n_mfcc)
        mfcc = mfcc.T

        if mfcc.shape[0] < target_length:
            padding = target_length - mfcc.shape[0]
            mfcc = np.pad(mfcc, ((0, padding), (0, 0)), mode='constant')
        elif mfcc.shape[0] > target_length:
            mfcc = mfcc[:target_length, :]

        return mfcc

    except Exception as e:
        print(f"Error processing {path}: {e}")
        return np.zeros((target_length, n_mfcc))

# ============================
# Label Encoding
# ============================
def label_to_onehot(label):
    onehot = np.zeros(num_species, dtype=np.float32)
    if label in species_to_index:
        onehot[species_to_index[label]] = 1.0
    return onehot

train_file_paths = [os.path.join(TRAIN_AUDIO_DIR, fname) for fname in sampled_df["filename"].values]
train_labels = [label_to_onehot(lbl) for lbl in sampled_df["primary_label"].values]

train_paths, val_paths, train_lab, val_lab = train_test_split(train_file_paths, train_labels, test_size=0.2, random_state=42)

# ============================
# Data Generator using MFCC Features with Augmentation
# ============================
def generator(paths, labels):
    for path, label in zip(paths, labels):
        mfcc = load_and_process_audio(path)
        yield mfcc, label

feature_dim = load_and_process_audio(train_paths[0]).shape
output_shapes = (feature_dim, (num_species,))
output_types = (tf.float32, tf.float32)

train_dataset = tf.data.Dataset.from_generator(lambda: generator(train_paths, train_lab), output_types=output_types, output_shapes=output_shapes).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_generator(lambda: generator(val_paths, val_lab), output_types=output_types, output_shapes=output_shapes).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# ============================
# Adaptive Dropout Class
# ============================
class AdaptiveDropout(callbacks.Callback):
    def __init__(self, model, initial_dropout=0.3, min_dropout=0.05, max_dropout=0.5,
                 scaling_factor=0.5, baseline_gap=0.05):
        super().__init__()
        self.model = model
        self.dropout_rate = initial_dropout
        self.min_dropout = min_dropout
        self.max_dropout = max_dropout
        self.scaling_factor = scaling_factor
        self.baseline_gap = baseline_gap

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        train_acc = logs.get('accuracy', 0)
        val_acc = logs.get('val_accuracy', 0)

        if train_acc > 0:
            relative_gap = (train_acc - val_acc) / train_acc
            powered_gap = relative_gap**2
        else:
            relative_gap = 0

        if powered_gap > self.baseline_gap:
            percent_change = self.scaling_factor * (((powered_gap)**.5 - self.baseline_gap) / self.baseline_gap)
            new_dropout = self.dropout_rate * (1 + percent_change)            
        else:
            percent_change = self.scaling_factor * ((self.baseline_gap - relative_gap) / self.baseline_gap)
            new_dropout = self.dropout_rate * (1 - percent_change)

        new_dropout = min(max(new_dropout, self.min_dropout), self.max_dropout)

        self.dropout_rate = new_dropout
        for layer in self.model.layers:
            if isinstance(layer, layers.Dropout):
                layer.rate = self.dropout_rate

        print(f"Epoch {epoch+1}: Training Acc = {train_acc:.4f}, Validation Acc = {val_acc:.4f}, "
              f"Relative Gap = {relative_gap:.4f}, New Dropout = {self.dropout_rate:.4f}")


# ============================
# WaveNet-cnn like Model Definition with Leaky ReLU and Dropout
# ============================
def create_wavenet_model(input_shape, num_classes, dropout_rate=DROPOUT_RATE):
    inputs = layers.Input(shape=input_shape)

    # CNN Layers for Spectral Feature Extraction
    cnn = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(layers.Reshape((input_shape[0], input_shape[1], 1))(inputs))
    cnn = layers.MaxPooling2D((2, 2))(cnn)
    cnn = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(cnn)
    cnn = layers.MaxPooling2D((2, 2))(cnn)
    cnn = layers.Reshape((-1, 64 * (input_shape[1] // 4)))(cnn) #Flattening for input into 1D conv layers

    # WaveNet-like Layers for Temporal Feature Extraction
    dilations = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
    filters = 64
    activation = 'leaky_relu'

    x = layers.Conv1D(32, 3, dilation_rate=1, padding='causal')(cnn)
    x = layers.LeakyReLU(alpha=alpha)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)

    for dilation in dilations:
        x = layers.Conv1D(filters, 3, dilation_rate=dilation, padding='causal')(x)
        x = layers.LeakyReLU(alpha=alpha)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout_rate)(x)

    x = layers.Conv1D(filters, 3, dilation_rate=512, padding='causal')(x)
    x = layers.LeakyReLU(alpha=alpha)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================
# Model Instantiation
# ============================
model = create_wavenet_model(((313, 42)), num_species)
adaptive_dropout_callback = AdaptiveDropout(model)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# ============================
# Training
# ============================
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=[early_stopping, adaptive_dropout_callback, reduce_lr])

# ============================
# Plot Training History
# ============================

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

# ============================
# Saving Model
# ============================
model.save("WAVENET256_Species_adaptive_dropout")
print("Model training complete. Saved as WAVENET256_Species_adaptive_dropout")


Epoch 1/100


NotFoundError: Graph execution error:

Detected at node 'model_2/conv2d_4/Relu' defined at (most recent call last):
    File "c:\Users\caspe\anaconda3\envs\SPIKEDETEC\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\caspe\anaconda3\envs\SPIKEDETEC\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\caspe\anaconda3\envs\SPIKEDETEC\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\caspe\anaconda3\envs\SPIKEDETEC\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\caspe\anaconda3\envs\SPIKEDETEC\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\caspe\anaconda3\envs\SPIKEDETEC\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\caspe\anaconda3\envs\SPIKED